#### Set container configuration

#### Read clean credit card data

In [0]:
credit_card_df = spark\
                .read\
                .format("csv")\
                .option("header",True)\
                .option("inferSchema",True)\
                .load("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/clean_data/")

display(credit_card_df)

index,City,Date,Card Type,Exp Type,Gender,Amount,cityname,countryname,month_value
0,"Delhi, India",29-Oct-14,Gold,Bills,F,82475,Delhi,India,Oct
1,"Greater Mumbai, India",22-Aug-14,Platinum,Bills,F,32555,Greater Mumbai,India,Aug
2,"Bengaluru, India",27-Aug-14,Silver,Bills,F,101738,Bengaluru,India,Aug
3,"Greater Mumbai, India",12-Apr-14,Signature,Bills,F,123424,Greater Mumbai,India,Apr
4,"Bengaluru, India",5-May-15,Gold,Bills,F,171574,Bengaluru,India,May
5,"Delhi, India",8-Sep-14,Silver,Bills,F,100036,Delhi,India,Sep
6,"Delhi, India",24-Feb-15,Gold,Bills,F,143250,Delhi,India,Feb
7,"Greater Mumbai, India",26-Jun-14,Platinum,Bills,F,150980,Greater Mumbai,India,Jun
8,"Delhi, India",28-Mar-14,Silver,Bills,F,192247,Delhi,India,Mar
9,"Delhi, India",1-Sep-14,Platinum,Bills,F,67932,Delhi,India,Sep


#### write a query to print top 5 cities with highest spends and their percentage contribution of total credit card spends

In [0]:
from pyspark.sql.functions import substring_index,col,sum,lit

total_amount = credit_card_df\
    .agg(sum("Amount").alias("total_amnt"))\
    .select("total_amnt")\
    .first()[0]

query_1_df = credit_card_df\
    .groupBy(col("cityname"))\
    .agg(sum("Amount").alias("sm_amnt"))\
    .withColumn("percentage",(col("sm_amnt")/total_amount)*100)\
    .orderBy(col("percentage").desc())\
    .limit(5)

query_1_df.write.format("csv").option("header",True).save("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/processed_data/query_1/")

#### write a query to print highest spend month and amount spent in that month for each card type

In [0]:
highest_month = credit_card_df\
    .groupBy("month_value")\
    .agg(sum("Amount").alias("amnt"))\
    .orderBy(col("amnt").desc())\
    .limit(1)\
    .select("month_value")\
    .first()[0]

query_2_df = credit_card_df\
    .filter(col("month_value")==highest_month)\
    .groupBy(col("Card Type"))\
    .agg(sum("Amount").alias("amnt"))

query_2_df.write.format("csv").option("header",True).save("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/processed_data/query_2/")

#### write a query to print the transaction details(all columns from the table) for each card type when it reaches a cumulative of 1000000 total spends(We should have 4 rows in the o/p one for each card type)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

query_3_df = credit_card_df\
    .withColumn("r_sum",sum("Amount").over(Window\
                                           .partitionBy(col("Card Type")).orderBy("Amount")))\
    .filter(col("r_sum") < 1000000)\
    .withColumn("drnk",dense_rank().over(Window\
                                         .partitionBy(col("Card Type")).orderBy(col("r_sum").desc())))\
    .filter(col("drnk")==1)\
    .drop("drnk")

query_3_df.write.format("csv").option("header",True).save("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/processed_data/query_3/")

#### write a query to find city which had lowest percentage spend for gold card type

In [0]:
query_4_df = credit_card_df\
    .filter(col("Card Type")=='Gold')\
    .groupBy(col("cityname"))\
    .agg(sum("Amount").alias("sm_amnt"))\
    .orderBy(col("sm_amnt"))\
    .limit(1)

query_4_df.write.format("csv").option("header",True).save("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/processed_data/query_4/")

#### write a query to print 3 columns: city, highest_expense_type , lowest_expense_type (example format : Delhi , bills, Fuel)

In [0]:
from pyspark.sql.functions import max,min
highest_expense = credit_card_df\
    .groupBy(col("cityname").alias("c_name"))\
    .agg(max("Amount").alias("max_amnt"))

highest_result = credit_card_df\
    .join(highest_expense,((credit_card_df.cityname == highest_expense.c_name)&(credit_card_df.Amount == highest_expense.max_amnt)),"inner")\
    .select("cityname",col("Exp Type").alias("highest_expense"))

lowest_expense = credit_card_df\
    .groupBy(col("cityname").alias("c_name"))\
    .agg(min("Amount").alias("min_amnt"))

lowest_result = credit_card_df\
    .join(lowest_expense,((credit_card_df.cityname == lowest_expense.c_name)&(credit_card_df.Amount == lowest_expense.min_amnt)),"inner")\
    .select(col("cityname").alias("cname"),col("Exp Type").alias("lowest_expense"))\

query_5_df = highest_result\
    .join(lowest_result,highest_result.cityname==lowest_result.cname)\
    .select("cityname","highest_expense","lowest_expense")

query_5_df.write.format("csv").option("header",True).save("abfss://cleanprocesseddata@bytestorageaccount.dfs.core.windows.net/processed_data/query_5/")

